#### Resource Link :- https://docs.langchain.com/oss/python/integrations/chat/openai

In [1]:
!pip install dotenv
import os
from dotenv import load_dotenv

load_dotenv(".env")



True

### Response 

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model= "gpt-4o-mini" , openai_api_key=os.getenv("OPENAI_API_KEY"))

messages = [
    ("system" , "you are the best cosulatant of politician to remind their weekness"),
    ("user" , "hii")
]
resp = llm.invoke(messages)
resp


/Users/rahul12.singh/Desktop/OPENAI/langchain_graph/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 27, 'total_tokens': 36, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bbc38b4db', 'id': 'chatcmpl-CzJgumrVEpKJZ1dDhkZsAdO7e0NwY', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--019bd081-f38d-7f12-9a49-7d49269c7db5-0', usage_metadata={'input_tokens': 27, 'output_tokens': 9, 'total_tokens': 36, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Streaming

In [3]:

stream = llm.stream(messages)

for chunk in stream:
    print(chunk.content)



Hello
!
 How
 can
 I
 assist
 you
 today
?




### Tools Calling

#### custom tools calling

In [4]:
%pip install langchain_core
from langchain_core.tools import tool
from typing import Dict , Any

## defining the tools
@tool
def GetWeather(location : str)-> Dict[str , Any]:
    """Get the current weather information for a given location. including temperature and conditions."""
    # Dummy implementation for illustration
    dummy_weather = {
        "Mumbai": {"temperature": 32, "condition": "Humid"},
        "Delhi": {"temperature": 35, "condition": "Hot"},
    }
    return {"location": location, "weather" : dummy_weather.get(location.lower(), {"error": "No data"})}

@tool
def GetPopulation(location : str)-> Dict[str , any]:
    """Get the current population for a given location."""
    # Dummy implementation for illustration
    dummy_population = {
        "Mumbai": 20411000,
        "Delhi": 16787941,
    }
    return {"location": location, "population" : dummy_population.get(location , "Unknown location")}


from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini" , openai_api_key=os.getenv("OPENAI_API_KEY"))

agent = llm.bind_tools([GetWeather , GetPopulation])

response = agent.invoke("weather of Delhi")

response
# response.tool_calls


Note: you may need to restart the kernel to use updated packages.


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_HNZNHZWb45qtjwIIUgvmcBzj', 'function': {'arguments': '{"location":"Delhi"}', 'name': 'GetWeather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 78, 'total_tokens': 92, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-CzJh371sXEaPsjCRYh1Ph9mrHXrii', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--019bd082-17d2-7dc0-beff-98201023dc34-0', tool_calls=[{'name': 'GetWeather', 'args': {'location': 'Delhi'}, 'id': 'call_HNZNHZWb45qtjwIIUgvmcBzj', 'type': 'tool_call'}], usage_metadata={'input_tokens': 78, 'output_tokens': 14, 'total_tokens': 92, 'input_token_details': {'

#### Agent Executer

In [16]:
# %pip install langchain_core
# %pip install langchain
# %pip install langchain_community
%pip install -U ddgs



from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from typing import Dict , Any
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate
from langchain_community.tools import DuckDuckGoSearchRun

WebSearchTool = DuckDuckGoSearchRun()



## defining the tools
@tool
def GetWeather(location : str)-> Dict[str , Any]:
    """Get the current weather information for a given location. including temperature and conditions."""
    # Dummy implementation for illustration
    dummy_weather = {
        "mumbai": {"temperature": 32, "condition": "Humid"},
        "delhi": {"temperature": 35, "condition": "Hot"},
    }
    return {"location": location, "weather" : dummy_weather.get(location.lower(), {"error": "No data"})}

@tool
def GetPopulation(location : str)-> Dict[str , any]:
    """Get the current population for a given location."""
    # Dummy implementation for illustration
    dummy_population = {
        "mumbai": 20411000,
        "delhi": 16787941,
    }
    return {"location": location, "population" : dummy_population.get(location , "Unknown location")}


prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. you have the access to tools whenever you need to use them."
    ""),
    ("user", "{input}"),
    ("assistant", "{agent_scratchpad}")
])


llm = ChatOpenAI(model="gpt-4o-mini" , openai_api_key=os.getenv("OPENAI_API_KEY"))

agent= create_tool_calling_agent( llm , [GetWeather , GetPopulation , WebSearchTool] , prompt=prompt)
agent_executor = AgentExecutor(agent = agent , tools= [GetWeather , GetPopulation, WebSearchTool], verbose=True)

response = agent_executor.invoke({"input": "tell about the Rahul kumar singh Nit jamshedpur"})

response
# response.tool_calls


Note: you may need to restart the kernel to use updated packages.


> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': 'Rahul Kumar Singh Nit Jamshedpur'}`




ValueError: Unsupported protocol version 0x304

#### builtin tools calling...

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini" , openai_api_key=os.getenv("OPENAI_API_KEY"))

buitInTool = {"type" : "web_search" }

agent =llm.bind_tools([buitInTool])
response = agent.invoke("current news yogi adityanath in hindi")
response.content

[{'type': 'text',
  'text': 'निम्नलिखित हालिया समाचार हैं जो मुख्यमंत्री योगी आदित्यनाथ से संबंधित हैं:\n\n1. **राष्ट्रीय परिषद में सदस्यता**: 14 दिसंबर 2025 को भाजपा के संगठन पर्व कार्यक्रम के दौरान, उत्तर प्रदेश से राष्ट्रीय परिषद के 120 सदस्यों की घोषणा की गई, जिसमें प्रधानमंत्री नरेंद्र मोदी, मुख्यमंत्री योगी आदित्यनाथ, रक्षा मंत्री राजनाथ सिंह, उप मुख्यमंत्री केशव प्रसाद मौर्य, स्मृति ईरानी, और अन्य शामिल हैं। ([yugwarta.com](https://www.yugwarta.com/Encyc/2025/12/14/National-Council-Member-Modi.html?utm_source=openai))\n\n2. **नए साल पर मुलाकातें**: 2 जनवरी 2026 को मुख्यमंत्री योगी आदित्यनाथ के सरकारी आवास पर नए साल के पहले दिन मंत्रियों, अधिकारियों, और अन्य गणमान्य व्यक्तियों की लंबी कतार देखी गई, जिन्होंने उनसे मुलाकात की और विकास तथा जनकल्याण पर चर्चा की। ([amritvichar.com](https://www.amritvichar.com/article/565896/a-queue-formed-to-meet-chief-minister-yogi-on-new-year-s-day--ministers-and-officials-met--and-discussions-on-development-and-public-welfare-were-held-openly?utm_s